In [ ]:
from pathlib import Path
from dotenv import load_dotenv
from multiprocessing import Pool
import time
from sqlalchemy import text, create_engine
import os
import pandas as pd
import json

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

pdf_files_folder = Path("//luxor/data/branch/Environmental Baseline Data\Version 4 - Final/PDF")
csv_tables_folder = Path("//luxor/data/branch/Environmental Baseline Data\Version 4 - Final/all_csvs")

if not pdf_files_folder.exists():
    print(pdf_files_folder, "does not exist!")
elif not csv_tables_folder.exists():
    print(csv_tables_folder, "does not exist!")
else:
    print("All paths are accessible.")


load_dotenv(override=True)
host = os.getenv("DB_HOST")
database = os.getenv("DB_DATABASE")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
engine_string = f"mysql+mysqldb://{user}:{password}@{host}/esa?charset=utf8"
engine = create_engine(engine_string)

In [ ]:
# CAREFUL! DELETES ALL CSV files and CSV DB entries, and resets PDFs (csvsExtracted = NULL)!
with engine.connect() as conn:
    result = conn.execute("DELETE FROM esa.csvs;")
    print(f"Deleted {result.rowcount} csvs from DB")
    result = conn.execute("UPDATE esa.pdfs SET csvsExtracted = NULL WHERE csvsExtracted IS NOT NULL;")
    print(f"Reset {result.rowcount} PDFs from DB (csvsExtracted = NULL)")
csvs = list(csv_tables_folder.glob("*.csv"))
for f in csvs:
    f.unlink()
print(f"Deleted {len(csvs)} CSV files")

In [ ]:
def create_args_for_csv_extraction():
    statement = text("SELECT * FROM esa.pdfs WHERE csvsExtracted IS NULL ORDER BY totalPages DESC;")
    with engine.connect() as conn:
        df = pd.read_sql(statement, conn)
    pdfs = df.to_dict("records")

    files = []
    for pdf in pdfs:
        pages = []
        for page in range (1, pdf["totalPages"] + 1):
            pages.append((pdf["pdfId"], page, engine_string, str(pdf_files_folder), str(csv_tables_folder)))
        files.append(pages)
    return files

In [ ]:
from external import extract_csv

log_file = "log.txt"
with Path(log_file).open("w") as f:
    pass # Clearing the log file

def log_it(s):
    with Path(log_file).open("a", encoding="utf-8-sig") as f:
        f.write(s)
    print(s)

start_time = time.time()
files = create_args_for_csv_extraction()[:]
time_stamp = time.strftime("%H:%M:%S %Y-%m-%d")
log_it(f"\nItems to process: {len(files)} at {time_stamp}\n")

for index, pages in enumerate(files):
    pdf_id = pages[0][0]
    time_stamp2 = time.strftime("%H:%M:%S %Y-%m-%d")
    log_it("\n--------------------------------------------------------")
    log_it(f"\n{pdf_id}: {len(pages)} pages to process at {time_stamp2}\n\n")
    start_time2 = time.time()
    with Pool() as pool:
        results = pool.map(extract_csv, pages)
    for result in results:
        log_it(result)

    with engine.connect() as conn:
        statement = text("UPDATE esa.pdfs SET csvsExtracted = :csvsExtracted WHERE pdfId = :pdfId;")
        result = conn.execute(statement, {"csvsExtracted": 'true', "pdfId": pdf_id})
    log_it(f"\n{pdf_id}: updated {result.rowcount} for {pdf_id} (csvsExtracted)\n")
    duration2 = round(time.time() - start_time2)
    log_it(f"\n{pdf_id}: done {len(pages)} pages in {duration2} seconds ({round(duration2/60, 2)} min)\n")

duration = round(time.time() - start_time)
log_it(f"\nDone {len(files)} items in {duration} seconds ({round(duration/60, 2)} min or {round(duration/3600, 2)} hours)")